# Language Agent Tree Search

<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/agent/lats_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[LATS (Language Agent Tree Search)](https://arxiv.org/pdf/2310.04406.pdf) by Zhou et al. combines LLM capabilities in planning, acting, and reasoning within a Monte Carlo tree search framework, allowing for deliberate and adaptive problem-solving guided by external feedback and self-reflection.

We've implemented this agent as a LlamaPack - you can either pip install it to run it out-of-the-box or call `download_llama_pack` to load the pack.

## Setup

In [59]:
%pip install llama-index-agent-lats
%pip install llama-index-core llama-index-readers-file
%pip install llama-index-llms-mistralai
%pip install llama-index-embeddings-huggingface
%pip install llama-index-embeddings-instructor


### Define Global Settings

In [60]:
import os

os.environ["MISTRAL_API_KEY"] = "iTDzYt5Htod7uyGjt5caLPfG899UQHju"

import nest_asyncio

nest_asyncio.apply()

In [61]:
from llama_index.llms.mistralai import MistralAI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

# NOTE: a higher temperate will help make the tree-expansion more diverse
llm = MistralAI(api_key=os.environ.get("MISTRAL_API_KEY"))
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

Settings.llm = llm
Settings.embed_model = embed_model

### Download Data

In [6]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

--2024-09-19 05:26:25--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1880483 (1.8M) [application/octet-stream]
Saving to: ‘data/10k/uber_2021.pdf’

data/10k/uber_2021. 100%[===================>]   1.79M  --.-KB/s    in 0.07s   

2024-09-19 05:26:25 (25.3 MB/s) - ‘data/10k/uber_2021.pdf’ saved [1880483/1880483]

--2024-09-19 05:26:25--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connec

In [63]:
import os
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    load_index_from_storage,
)
from llama_index.core.storage import StorageContext


if not os.path.exists("./storage/lyft"):
    # load data
    lyft_docs = SimpleDirectoryReader(
        input_files=["./data/10k/lyft_2021.pdf"]
    ).load_data()
    uber_docs = SimpleDirectoryReader(
        input_files=["./data/10k/uber_2021.pdf"]
    ).load_data()

    # build index
    lyft_index = VectorStoreIndex.from_documents(lyft_docs)
    uber_index = VectorStoreIndex.from_documents(uber_docs)

    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")
else:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/lyft"
    )
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/uber"
    )
    uber_index = load_index_from_storage(storage_context)

### Setup Tools

In [65]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

In [66]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool. "
                "The input is used to power a semantic search engine."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool. "
                "The input is used to power a semantic search engine."
            ),
        ),
    ),
]

## Setup Agent

Now we can setup the LATS agent.

Here, `num_expansions` refers to the number of possible sub-actions to explore under each node. `num_expansions=2` means we will explore to possible next-actions for every parent action.

`max_rollouts` refers to how deep each exploration of the search space continues. `max_rollouts=5` means a maximum depth of 5 is explored in the tree.

In [67]:
from llama_index.agent.lats import LATSAgentWorker


agent_worker = LATSAgentWorker.from_tools(
    query_engine_tools,
    llm=llm,
    num_expansions=2,
    max_rollouts=3,  # using -1 for unlimited rollouts
    verbose=True,
)
agent = agent_worker.as_agent()


## Run Some Queries

First, lets use step-wise execution and the lower-level API to create and execute a task.

In [70]:
task = agent.create_task(
    "Given the risk factors of Uber and Lyft described in their 10K files, "
    "which company is performing better? Please use concrete numbers to inform your decision."
)

In [71]:
# run initial step
step_output = agent.run_step(task.task_id)

> Selecting node to expand: Observation: Given the risk factors of Uber and Lyft described in their 10K files, which company is performing better? Please use concrete numbers to inform your decision.
> Got candidates: ['Retrieve the 10K files of Uber and Lyft.', 'Analyze the risk factors described in the 10K files of Uber and Lyft.']
=== Calling Function ===
Calling function: lyft_10k with args: {"input": "What are the risk factors described in Lyft's 10K for the year 2021?"}
=== Calling Function ===
Calling function: uber_10k with args: {"input": "Provide the financial performance of Uber for the year 2021."}
=== Function Output ===
Lyft's 10K for the year 2021 describes various risk factors, including general economic factors such as the impact of the COVID-19 pandemic, natural disasters, economic downturns, public health crises, and political crises, as well as general macroeconomic conditions. Operational factors include Lyft's limited operating history, financial performance and t

From the step output, we can inspect the task state

In [33]:
for step in (
    step_output.task_step.step_state["root_node"].children[0].current_reasoning
):
    print(step)
    print("---------")

observation='Given the risk factors of Uber and Lyft described in their 10K files, which company is performing better? Please use concrete numbers to inform your decision.' return_direct=False
---------
observation='Retrieve the 10K files of Uber and Lyft.' return_direct=False
---------


In [36]:
for step in (
    step_output.task_step.step_state["root_node"]
    .children[0]
    .children[0]
    .current_reasoning
):
    print(step)
    print("---------")

observation='Given the risk factors of Uber and Lyft described in their 10K files, which company is performing better? Please use concrete numbers to inform your decision.' return_direct=False
---------
observation='Retrieve the 10K files of Uber and Lyft.' return_direct=False
---------
thought='The current language of the user is English. I need to use tools to retrieve the 10K files of Uber and Lyft to compare their performance.' action='uber_10k' action_input={'input': 'Provide the financial performance of Uber for the year 2021.'}
---------
observation="For the year ended December 31, 2021, Uber reported a net loss including non-controlling interests of $570 million. The company's operations and support expenses increased by 3% to $1,877 million, while sales and marketing expenses increased by 34% to $4,789 million. Research and development expenses decreased by 7% to $2,054 million, and general and administrative expenses decreased by 13% to $2,316 million. Depreciation and amorti

Lets loop until we finish the task

In [ ]:
# repeat until the last step is reached
while not step_output.is_last:
    step_output = agent.run_step(task.task_id)

response = agent.finalize_response(task.task_id)

> Selecting node to expand: Observation: Uber Technologies, Inc., in its 2021 financial statements, consolidates its wholly-owned and majority-owned subsidiaries, as well as variable interest entities where it is the primary beneficiary. The financial statements are prepared in accordance with GAAP, and management uses estimates and assumptions that affect reported financial figures, such as the fair values of investments, useful lives of assets, and reserves for income taxes and insurance, among others. These estimates consider the impact of the COVID-19 pandemic on market data and investment recoverability.

Key financial risks for Uber include concentration of credit risk, where cash and other receivables are potentially subject to credit risk concentration. The company's cash, cash equivalents, and securities consist largely of high-credit-quality money market funds, U.S. government and agency securities, and corporate debt securities. Despite exceeding insured limits, these are pl

In [ ]:
print(str(response))

Based on the information provided from the 10K files of Uber and Lyft for 2021, here is a comparative analysis of their financial data and risk factors:

**Financial Data:**
1. **Uber** did not disclose specific revenue figures in the provided information, focusing more on their financial management practices and the quality of their financial instruments. They emphasized the high credit quality of their cash equivalents and securities, and the use of GAAP-compliant financial statements influenced by estimates considering the COVID-19 impact.
   
2. **Lyft** reported total revenue of $3,208,323,000 for 2021, with a significant drop due to the COVID-19 pandemic and a partial recovery, still below pre-pandemic levels. This gives a clear numeric insight into their financial status during the year.

**Risk Factors:**
1. **Uber's** key risk factors include concentration of credit risk, with their financial assets placed in high-credit-quality institutions, and various uncertainties that cou

Using `.chat()` directly, we can have the agent automatically run until completion.

In [ ]:
agent.reset()

response = agent.chat(
    "Given the revenue growth and risk factors of Uber and Lyft, "
    "which company is performing better? Please use concrete numbers to inform your decision."
)

> Selecting node to expand: Observation: Given the revenue growth and risk factors of Uber and Lyft, which company is performing better? Please use concrete numbers to inform your decision.
> Got candidates: ['Review financial reports of Uber and Lyft for the latest fiscal year to compare revenue growth.', 'Analyze risk factors mentioned in the latest quarterly reports of Uber and Lyft.']
=== Calling Function ===
Calling function: uber_10k with args: {"input": "What was Uber's revenue growth for the fiscal year 2021?"}
=== Function Output ===
Uber's revenue for the fiscal year 2021 was $17,455 million, which represents a growth of 57% compared to the revenue of $11,139 million in 2020.
> Generated new reasoning step: Thought: The current language of the user is English. I need to use tools to gather data on the revenue growth of Uber and Lyft for the latest fiscal year, which is 2021.
Action: uber_10k
Action Input: {'input': "What was Uber's revenue growth for the fiscal year 2021?"}
O

In [58]:
print(str(response))

Observation: In 2021, Uber's key financial metrics include:

- **Revenue**: $17,455 million
- **Cost of Revenue (excluding depreciation and amortization)**: $9,351 million, which is 54% of revenue
- **Operations and Support Expenses**: $1,877 million, which is 11% of revenue
- **Sales and Marketing Expenses**: $4,789 million, which is 27% of revenue
- **Research and Development Expenses**: $2,054 million, which is 12% of revenue
- **General and Administrative Expenses**: $2,316 million, which is 13% of revenue
- **Depreciation and Amortization**: $902 million, which is 5% of revenue
- **Net Loss including Non-controlling Interests**: $(570) million
- **Net Cash Used in Operating Activities**: $(445) million
- **Net Cash Used in Investing Activities**: $(1,201) million


In [ ]:

import requests
import json

url = "http://64.247.196.19:8000/v1/chat/completions"

headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer token"
}

data = {
    "model": "mistralai/Mistral-Small-Instruct-2409",
    "messages": [
        {
            "role": "system",
            "content": "You are a helpful assistant."
        },
        {
            "role": "user",
            "content": "How often does the letter r occur in Mistral?"
        }
    ]
}

response = requests.post(url, headers=headers, data=json.dumps(data))

print(response.status_code)
print(response.json())
